In [50]:
from carbon import CarbonSimulatorUI, P, __version__, __date__
from math import sqrt
print(f"Carbon v{__version__} ({__date__})")
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonSimulatorUI))

Carbon v2.0-beta2 (8/Dec/2022)
CarbonSimulatorUI v2.2 (8/Dec/2022)


# Carbon Simulation - Test 36 - Liquidity Approximation

In [51]:
Sim = CarbonSimulatorUI(verbose=False, raiseonerror=False, pair="ETH/USDC")
Sim

CarbonSimulatorUI(<0 orders, 0 trades>, pair='ETH/USDC', mm='exact', xf=True)

In [52]:
Sim.add_order("ETH", 10, 2000, 3000)
Sim.add_order("ETH", 10, 2500, 2500)
Sim.add_order("USDC", 10*1250, 1500, 1000)
Sim.add_order("USDC", 10*1250, 1250, 1250)
Sim

CarbonSimulatorUI(<8 orders, 0 trades>, pair='ETH/USDC', mm='exact', xf=True)

In [53]:
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,10.0,10.0,ETH,False,2000.0,3000.0,2000.0,USDC per ETH,1
1,1,ETHUSDC,USDC,0.0,0.0,USDC,True,None,None,None,USDC per ETH,0
2,2,ETHUSDC,ETH,10.0,10.0,ETH,False,2500.0,2500.0,2500.0,USDC per ETH,3
3,3,ETHUSDC,USDC,0.0,0.0,USDC,True,None,None,None,USDC per ETH,2
4,4,ETHUSDC,USDC,12500.0,12500.0,USDC,False,1500.0,1000.0,1500.0,USDC per ETH,5
5,5,ETHUSDC,ETH,0.0,0.0,ETH,True,None,None,None,USDC per ETH,4
6,6,ETHUSDC,USDC,12500.0,12500.0,USDC,False,1250.0,1250.0,1250.0,USDC per ETH,7
7,7,ETHUSDC,ETH,0.0,0.0,ETH,True,None,None,None,USDC per ETH,6


In [54]:
Sim.state().keys()

dict_keys(['orders', 'orderuis', 'trades'])

In [55]:
Sim.state()["orderuis"]

{0: CarbonOrderUI(pair=CarbonPair(slashpair='ETH/USDC', tknb='ETH', tknq='USDC'), tkn='ETH', B=0.018257418583505537, S=0.004103261191492361, yint=10.0, y=10.0),
 1: CarbonOrderUI(pair=CarbonPair(slashpair='ETH/USDC', tknb='ETH', tknq='USDC'), tkn='USDC', B=0.0, S=0.0, yint=0.0, y=0.0),
 2: CarbonOrderUI(pair=CarbonPair(slashpair='ETH/USDC', tknb='ETH', tknq='USDC'), tkn='ETH', B=0.02, S=0.0, yint=10.0, y=10.0),
 3: CarbonOrderUI(pair=CarbonPair(slashpair='ETH/USDC', tknb='ETH', tknq='USDC'), tkn='USDC', B=0.0, S=0.0, yint=0.0, y=0.0),
 4: CarbonOrderUI(pair=CarbonPair(slashpair='ETH/USDC', tknb='ETH', tknq='USDC'), tkn='USDC', B=31.622776601683793, S=7.107056860390376, yint=12500.0, y=12500.0),
 5: CarbonOrderUI(pair=CarbonPair(slashpair='ETH/USDC', tknb='ETH', tknq='USDC'), tkn='ETH', B=0.0, S=0.0, yint=0.0, y=0.0),
 6: CarbonOrderUI(pair=CarbonPair(slashpair='ETH/USDC', tknb='ETH', tknq='USDC'), tkn='USDC', B=35.35533905932738, S=0.0, yint=12500.0, y=12500.0),
 7: CarbonOrderUI(pair=

In [56]:
help(Sim.state()["orderuis"][0].liquidity_approx)

Help on method liquidity_approx in module carbon.carbon_order_ui:

liquidity_approx(price1, price2, tkn=None, asperc=False) method of carbon.carbon_order_ui.CarbonOrderUI instance
    returns the approximate liquidity between start and end, in tkn
    
    :price1/2:      the start and end price of the range (in any order; quoted in price convention of pair)
    :tkn:           the token in which the liquidity is quoted (if None: base token)
    :asperc:        if True, return percentage total liquidity rather than tkn number; default is False
    :returns:       the liquidity in [price1, price2], quoted in tkn (or percent)



## Order 0 ETH range

In [57]:
oui = Sim.state()["orderuis"][0]
assert (oui.pmin, oui.pmax, oui.total_liquidity) == (2000.0, 3000.0, (10.0, 'ETH'))
oui.pmin, oui.pmax, oui.total_liquidity

(2000.0, 3000.0, (10.0, 'ETH'))

In [58]:
r = oui.liquidity_approx(500, 600, "ETH", asperc=True)
assert r == 0
r

0

In [59]:
r = oui.liquidity_approx(3000, 3020, "ETH", asperc=True)
assert r == 0
r

0

In [60]:
r = oui.liquidity_approx(2000, 3000, "ETH", asperc=True)
assert r == 1.
r

1.0

In [61]:
r = oui.liquidity_approx(3000, 2000, "ETH", asperc=True)
assert r == 1
r

1.0

In [62]:
r = oui.liquidity_approx(2000, 2500, "ETH", asperc=True)
assert r == 0.5
r

0.5

In [63]:
r = oui.liquidity_approx(2500, 3000, "ETH", asperc=True)
assert r == 0.5
r

0.5

In [64]:
r = oui.liquidity_approx(2250, 2750, "ETH", asperc=True)
assert r == 0.5
r

0.5

In [65]:
r = oui.liquidity_approx(2000, 3000, "ETH", asperc=False)
assert r == 10.
r

10.0

In [66]:
r = oui.liquidity_approx(2000, 2500, "ETH", asperc=False)
assert r == 5.
r

5.0

In [67]:
r = oui.liquidity_approx(2000, 2500, asperc=False)
assert r == 5.
r

5.0

In [68]:
r = oui.liquidity_approx(2000, 3000, "USDC", asperc=False)
assert int(r) == int(10*sqrt(2000*3000))
r, sqrt(2000*3000)

(24494.89742783178, 2449.489742783178)

In [69]:
r = oui.liquidity_approx(2250, 2750, "USDC", asperc=False)
assert int(r) == int(10*sqrt(2250*2750)*0.5)
r, sqrt(2250*2750)*0.5

(12437.34296383275, 1243.734296383275)

## Order 1 ETH point

In [70]:
oui = Sim.state()["orderuis"][2]
oui.pmin, oui.pmax, oui.total_liquidity

(2500.0, 2500.0, (10.0, 'ETH'))

In [71]:
r = oui.liquidity_approx(2000, 2499, "ETH", asperc=True)
assert r == 0
r

0

In [72]:
r = oui.liquidity_approx(2501, 3020, "ETH", asperc=True)
assert r == 0
r

0

In [73]:
r = oui.liquidity_approx(2499, 2501, "ETH", asperc=True)
assert r == 1
r

1.0

In [74]:
r = oui.liquidity_approx(2500, 2501, "ETH", asperc=True)
assert r == 1
r

1.0

In [75]:
r = oui.liquidity_approx(2499, 2500, "ETH", asperc=True)
assert r == 0
r

0

In [76]:
r = oui.liquidity_approx(2500, 2500, "ETH", asperc=True)
assert r == 1
r

1.0

In [77]:
r = oui.liquidity_approx(2499, 2501, "ETH", asperc=False)
assert r == 10
r

10.0

In [78]:
r = oui.liquidity_approx(2499, 2501, "USDC", asperc=False)
assert r == 25000
r

25000.0

In [79]:
r = oui.liquidity_approx(2500, 2501, "USDC", asperc=False)
assert r == 25000
r

25000.0

In [80]:
r = oui.liquidity_approx(2500, 2500, "USDC", asperc=False)
assert r == 25000
r

25000.0

In [81]:
r = oui.liquidity_approx(2000, 3000, "USDC", asperc=False)
assert r == 25000
r

25000.0

## Order 2 USDC range

In [82]:
oui = Sim.state()["orderuis"][4]
oui.pmin, oui.pmax, oui.total_liquidity

(1000.0, 1500.0, (12500.0, 'USDC'))

In [83]:
r = oui.liquidity_approx(500, 600, "USDC", asperc=False)
assert r == 0
r

0

In [84]:
r = oui.liquidity_approx(1600, 2000, "USDC", asperc=False)
assert r == 0
r

0

In [85]:
r = oui.liquidity_approx(1000, 1500, "USDC", asperc=False)
assert r == 12500
r

12500.0

In [86]:
r = oui.liquidity_approx(1000, 1100, "USDC", asperc=False)
assert r == 2500
r

2500.0

In [87]:
r = oui.liquidity_approx(1400, 1500, "USDC", asperc=False)
assert r == 2500
r

2500.0

In [88]:
r = oui.liquidity_approx(1000, 1100, "ETH", asperc=False)
assert r == 2500/sqrt(1000*1100)
r, 2500/sqrt(1000*1100)

(2.3836564731139807, 2.3836564731139807)

In [89]:
r = oui.liquidity_approx(1400, 1500, "ETH", asperc=False)
assert int(r) == int(2500/sqrt(1400*1500))
r, 2500/sqrt(1400*1500)

(1.7251638983558857, 1.7251638983558855)

## Order 3 USDC point

In [90]:
oui = Sim.state()["orderuis"][6]
oui.pmin, oui.pmax, oui.total_liquidity

(1250.0, 1250.0, (12500.0, 'USDC'))

In [91]:
r = oui.liquidity_approx(1249, 1251, "USDC", asperc=False)
assert r == 12500
r

12500.0

In [92]:
r = oui.liquidity_approx(1000, 2000, "USDC", asperc=False)
assert r == 12500
r

12500.0

In [93]:
r = oui.liquidity_approx(1249, 1250, "USDC", asperc=False)
assert r == 0
r

0

In [94]:
r = oui.liquidity_approx(1250, 1251, "USDC", asperc=False)
assert r == 12500
r

12500.0

In [95]:
r = oui.liquidity_approx(1249, 1251, "ETH", asperc=False)
assert r == 10
r

10.0

In [96]:
r = oui.liquidity_approx(1000, 2000, "ETH", asperc=False)
assert r == 10
r

10.0

In [97]:
r = oui.liquidity_approx(1249, 1250, "ETH", asperc=False)
assert r == 0
r

0

In [98]:
r = oui.liquidity_approx(1250, 1251, "ETH", asperc=False)
assert r == 10
r

10.0